In [10]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gp

In [2]:
df = xr.open_dataset("../heatmap_data/heatmap.nc")

In [4]:
chlor = pd.DataFrame(df['chlor_a'].to_pandas())

In [6]:
chlor_fixed = chlor.stack().reset_index()

In [8]:
chlor_fixed.columns = ['lat', 'lon', 'chlor']

In [9]:
chlor_fixed

,lat,lon,chlor
0,83.979164,-32.770828,0.589780
1,83.979164,-32.729164,0.589780
2,83.979164,-32.687496,0.589780
3,83.979164,-32.645828,0.589780
4,83.979164,-32.604164,0.589780
...,...,...,...
13654740,-53.645836,-159.104172,0.239841
13654741,-53.645836,-159.062500,0.239841
13654742,-53.687504,-159.104172,0.256716
13654743,-53.687504,-159.062500,0.256716


In [19]:
shp = gp.read_file('../great_lakes/hydro_p_Lake.shp').geometry[0]

In [21]:
minLon, minLat, maxLon, maxLat = shp.bounds

In [22]:
df_filtered = chlor_fixed.loc[((chlor_fixed['lat'] < maxLat) & (chlor_fixed['lat'] > minLat)) & ((chlor_fixed['lon'] < maxLon) & (chlor_fixed['lon'] > minLon))]

In [24]:
chlor_gdf = gp.GeoDataFrame(df_filtered, geometry = gp.points_from_xy(df_filtered.lon, df_filtered.lat))

In [25]:
chlor_gdf

,lat,lon,chlor,geometry
2231035,48.895832,-88.229164,5.697269,POINT (-88.22916 48.89583)
2231036,48.895832,-88.187500,5.697268,POINT (-88.18750 48.89583)
2231037,48.895832,-88.145828,6.636189,POINT (-88.14583 48.89583)
2231038,48.895832,-88.104164,6.491833,POINT (-88.10416 48.89583)
2231039,48.895832,-88.062500,6.096063,POINT (-88.06250 48.89583)
...,...,...,...,...
2797187,41.437500,-82.354164,11.554067,POINT (-82.35416 41.43750)
2797188,41.437500,-82.312500,10.545052,POINT (-82.31250 41.43750)
2797189,41.437500,-82.270828,10.545052,POINT (-82.27083 41.43750)
2801437,41.395832,-82.520828,16.284721,POINT (-82.52083 41.39583)


In [26]:
chlor_clipped = gp.clip(chlor_gdf, shp)

In [27]:
chlor_clipped

,lat,lon,chlor,geometry
2231035,48.895832,-88.229164,5.697269,POINT (-88.22916 48.89583)
2231036,48.895832,-88.187500,5.697268,POINT (-88.18750 48.89583)
2231037,48.895832,-88.145828,6.636189,POINT (-88.14583 48.89583)
2231038,48.895832,-88.104164,6.491833,POINT (-88.10416 48.89583)
2231039,48.895832,-88.062500,6.096063,POINT (-88.06250 48.89583)
...,...,...,...,...
2797187,41.437500,-82.354164,11.554067,POINT (-82.35416 41.43750)
2797188,41.437500,-82.312500,10.545052,POINT (-82.31250 41.43750)
2797189,41.437500,-82.270828,10.545052,POINT (-82.27083 41.43750)
2801437,41.395832,-82.520828,16.284721,POINT (-82.52083 41.39583)


In [29]:
final_df = chlor_clipped.drop(columns = 'geometry')

In [30]:
final_df

,lat,lon,chlor
2231035,48.895832,-88.229164,5.697269
2231036,48.895832,-88.187500,5.697268
2231037,48.895832,-88.145828,6.636189
2231038,48.895832,-88.104164,6.491833
2231039,48.895832,-88.062500,6.096063
...,...,...,...
2797187,41.437500,-82.354164,11.554067
2797188,41.437500,-82.312500,10.545052
2797189,41.437500,-82.270828,10.545052
2801437,41.395832,-82.520828,16.284721


In [38]:
final_df.to_csv('../heatmap_data/heatmap_csv.csv', index = False)

In [32]:
final_json = []
for row in final_df.itertuples(index = False):
    lat, lon, chlor = row[0], row[1], row[2]
    final_json.append([lat, lon, chlor])

In [33]:
final_json

[[48.89583206176758, -88.22916412353516, 5.697268962860107],
 [48.89583206176758, -88.1875, 5.697268486022949],
 [48.89583206176758, -88.14582824707031, 6.6361894607543945],
 [48.89583206176758, -88.10416412353516, 6.491832733154297],
 [48.89583206176758, -88.0625, 6.096063137054443],
 [48.89583206176758, -87.97916412353516, 4.79927396774292],
 [48.89583206176758, -87.9375, 4.929152011871338],
 [48.89583206176758, -87.89582824707031, 5.3164777755737305],
 [48.89583206176758, -87.85416412353516, 5.421134948730469],
 [48.89583206176758, -87.8125, 5.10658073425293],
 [48.89583206176758, -87.77082824707031, 6.323242664337158],
 [48.89583206176758, -87.72916412353516, 6.865440845489502],
 [48.854164123535156, -88.1875, 6.3643670082092285],
 [48.854164123535156, -88.14582824707031, 6.3643670082092285],
 [48.854164123535156, -88.10416412353516, 5.5485310554504395],
 [48.854164123535156, -88.0625, 5.326412677764893],
 [48.854164123535156, -88.02082824707031, 3.9188287258148193],
 [48.854164123

In [37]:
import json
with open('../heatmap_data/heatmap_data.json', 'w') as f:
    json.dump(final_json, f)